In [1]:
import pandas as pd
import numpy as np

## Read the CSV

In [2]:
# loading the dataset
df = pd.read_csv("resources/train_transformed_2.csv")
df

,Survived,Pclass,Age,SibSp,Parch,Fare,Family_Size,Age*Class,Fare_Per_Person,Sex_female,...,Title_Mrs,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Deck_U
0,0,3,22,1,0,7.2500,1,66,3.62500,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,38,1,0,71.2833,1,38,35.64165,1,...,1,0,0,1,0,0,0,0,0,0
2,1,3,26,0,0,7.9250,0,78,7.92500,1,...,0,0,0,0,0,0,0,0,0,1
3,1,1,35,1,0,53.1000,1,35,26.55000,1,...,1,0,0,1,0,0,0,0,0,0
4,0,3,35,0,0,8.0500,0,105,8.05000,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,0,2,27,0,0,13.0000,0,54,13.00000,0,...,0,0,0,0,0,0,0,0,0,1
885,1,1,19,0,0,30.0000,0,19,30.00000,1,...,0,0,1,0,0,0,0,0,0,0
886,0,3,17,1,2,23.4500,3,51,5.86250,1,...,0,0,0,0,0,0,0,0,0,1
887,1,1,26,0,0,30.0000,0,26,30.00000,0,...,0,0,0,1,0,0,0,0,0,0


## Select features

In [3]:
# Set features. This will also be used as your x values.
selected_features = df[["Pclass", "Age", "Fare", "Family_Size", "Age*Class", "Fare_Per_Person", "Sex_female", "Title_Mr"]]
feature_names = selected_features.columns

## Create a Train Test Split

In [4]:
# assign X(data) and y(target)
data = selected_features
target = df["Survived"]
print(data.shape, target.shape)

(889, 8) (889,)


In [5]:
# split the data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [6]:
X_train

,Pclass,Age,Fare,Family_Size,Age*Class,Fare_Per_Person,Sex_female,Title_Mr
375,3,22,7.2500,0,66,7.2500,1,0
457,2,50,10.5000,0,100,10.5000,1,0
731,2,36,0.0000,0,72,0.0000,0,1
506,1,20,26.5500,0,20,26.5500,0,1
828,3,15,14.4542,1,45,7.2271,1,0
...,...,...,...,...,...,...,...,...
106,3,35,7.7750,0,105,7.7750,0,1
270,3,25,0.0000,0,75,0.0000,0,1
860,1,48,25.9292,0,48,25.9292,1,0
435,3,21,34.3750,4,63,6.8750,1,0


## Pre-processing

In [7]:
# Scale your data
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Train the model

In [8]:
from sklearn.svm import SVC
model = SVC(kernel='linear')
model

SVC(kernel='linear')

In [9]:
# fit(train) our model using the training data
model.fit(X_train_scaled, y_train)

SVC(kernel='linear')

In [10]:
# validate the model using the test data
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.8393393393393394
Testing Data Score: 0.8071748878923767


In [11]:
pred_df = pd.DataFrame({"Predictions": model.predict(X_test_scaled), "Actual": y_test})
pred_df

,Predictions,Actual
280,0,0
434,1,1
39,1,1
417,0,0
584,1,1
...,...,...
424,0,0
101,0,0
199,0,0
398,1,1


## Hyperparameter Tuning

In [12]:
# Create the GridSearchCV model/estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'random_state': [0, 42],
              'tol': [1, 100, 1000, 10000, 10000, 1000000, 10000000],
              'degree': [1, 2, 3],
              'kernel': ['linear', 'poly', 'rbf'],
              'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [13]:
# Train the model with GridSearch estimator...
# this will take the model and try each combination of parameters
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 2016 candidates, totalling 10080 fits
[CV] C=1, degree=1, gamma=0.0001, kernel=linear, random_state=0, tol=1 
[CV]  C=1, degree=1, gamma=0.0001, kernel=linear, random_state=0, tol=1, score=0.828, total=   0.0s
[CV] C=1, degree=1, gamma=0.0001, kernel=linear, random_state=0, tol=1 
[CV]  C=1, degree=1, gamma=0.0001, kernel=linear, random_state=0, tol=1, score=0.865, total=   0.0s
[CV] C=1, degree=1, gamma=0.0001, kernel=linear, random_state=0, tol=1 
[CV]  C=1, degree=1, gamma=0.0001, kernel=linear, random_state=0, tol=1, score=0.805, total=   0.0s
[CV] C=1, degree=1, gamma=0.0001, kernel=linear, random_state=0, tol=1 
[CV]  C=1, degree=1, gamma=0.0001, kernel=linear, random_state=0, tol=1, score=0.842, total=   0.0s
[CV] C=1, degree=1, gamma=0.0001, kernel=linear, random_state=0, tol=1 
[CV]  C=1, degree=1, gamma=0.0001, kernel=linear, random_state=0, tol=1, score=0.835, total=   0.0s
[CV] C=1, degree=1, gamma=0.0001, kernel=linear, random_state=0, tol=100 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  C=1, degree=1, gamma=0.0001, kernel=poly, random_state=0, tol=1000, score=0.391, total=   0.0s
[CV] C=1, degree=1, gamma=0.0001, kernel=poly, random_state=0, tol=1000 
[CV]  C=1, degree=1, gamma=0.0001, kernel=poly, random_state=0, tol=1000, score=0.383, total=   0.0s
[CV] C=1, degree=1, gamma=0.0001, kernel=poly, random_state=0, tol=1000 
[CV]  C=1, degree=1, gamma=0.0001, kernel=poly, random_state=0, tol=1000, score=0.383, total=   0.0s
[CV] C=1, degree=1, gamma=0.0001, kernel=poly, random_state=0, tol=10000 
[CV]  C=1, degree=1, gamma=0.0001, kernel=poly, random_state=0, tol=10000, score=0.388, total=   0.0s
[CV] C=1, degree=1, gamma=0.0001, kernel=poly, random_state=0, tol=10000 
[CV]  C=1, degree=1, gamma=0.0001, kernel=poly, random_state=0, tol=10000, score=0.391, total=   0.0s
[CV] C=1, degree=1, gamma=0.0001, kernel=poly, random_state=0, tol=10000 
[CV]  C=1, degree=1, gamma=0.0001, kernel=poly, random_state=0, tol=10000, score=0.391, total=   0.0s
[CV] C=1, degree=1, gam

[CV] C=1, degree=1, gamma=0.0001, kernel=rbf, random_state=0, tol=1000000 
[CV]  C=1, degree=1, gamma=0.0001, kernel=rbf, random_state=0, tol=1000000, score=0.383, total=   0.0s
[CV] C=1, degree=1, gamma=0.0001, kernel=rbf, random_state=0, tol=1000000 
[CV]  C=1, degree=1, gamma=0.0001, kernel=rbf, random_state=0, tol=1000000, score=0.383, total=   0.0s
[CV] C=1, degree=1, gamma=0.0001, kernel=rbf, random_state=0, tol=10000000 
[CV]  C=1, degree=1, gamma=0.0001, kernel=rbf, random_state=0, tol=10000000, score=0.388, total=   0.0s
[CV] C=1, degree=1, gamma=0.0001, kernel=rbf, random_state=0, tol=10000000 
[CV]  C=1, degree=1, gamma=0.0001, kernel=rbf, random_state=0, tol=10000000, score=0.391, total=   0.0s
[CV] C=1, degree=1, gamma=0.0001, kernel=rbf, random_state=0, tol=10000000 
[CV]  C=1, degree=1, gamma=0.0001, kernel=rbf, random_state=0, tol=10000000, score=0.391, total=   0.0s
[CV] C=1, degree=1, gamma=0.0001, kernel=rbf, random_state=0, tol=10000000 
[CV]  C=1, degree=1, gamma=0

[CV] C=1, degree=1, gamma=0.0005, kernel=linear, random_state=42, tol=100 
[CV]  C=1, degree=1, gamma=0.0005, kernel=linear, random_state=42, tol=100, score=0.388, total=   0.0s
[CV] C=1, degree=1, gamma=0.0005, kernel=linear, random_state=42, tol=100 
[CV]  C=1, degree=1, gamma=0.0005, kernel=linear, random_state=42, tol=100, score=0.391, total=   0.0s
[CV] C=1, degree=1, gamma=0.0005, kernel=linear, random_state=42, tol=100 
[CV]  C=1, degree=1, gamma=0.0005, kernel=linear, random_state=42, tol=100, score=0.391, total=   0.0s
[CV] C=1, degree=1, gamma=0.0005, kernel=linear, random_state=42, tol=100 
[CV]  C=1, degree=1, gamma=0.0005, kernel=linear, random_state=42, tol=100, score=0.383, total=   0.0s
[CV] C=1, degree=1, gamma=0.0005, kernel=linear, random_state=42, tol=100 
[CV]  C=1, degree=1, gamma=0.0005, kernel=linear, random_state=42, tol=100, score=0.383, total=   0.0s
[CV] C=1, degree=1, gamma=0.0005, kernel=linear, random_state=42, tol=1000 
[CV]  C=1, degree=1, gamma=0.0005,

...

[CV]  C=50, degree=3, gamma=0.005, kernel=linear, random_state=0, tol=100, score=0.388, total=   0.0s
[CV] C=50, degree=3, gamma=0.005, kernel=linear, random_state=0, tol=100 
[CV]  C=50, degree=3, gamma=0.005, kernel=linear, random_state=0, tol=100, score=0.391, total=   0.0s
[CV] C=50, degree=3, gamma=0.005, kernel=linear, random_state=0, tol=100 
[CV]  C=50, degree=3, gamma=0.005, kernel=linear, random_state=0, tol=100, score=0.391, total=   0.0s
[CV] C=50, degree=3, gamma=0.005, kernel=linear, random_state=0, tol=100 
[CV]  C=50, degree=3, gamma=0.005, kernel=linear, random_state=0, tol=100, score=0.383, total=   0.0s
[CV] C=50, degree=3, gamma=0.005, kernel=linear, random_state=0, tol=100 
[CV]  C=50, degree=3, gamma=0.005, kernel=linear, random_state=0, tol=100, score=0.383, total=   0.0s
[CV] C=50, degree=3, gamma=0.005, kernel=linear, random_state=0, tol=1000 
[CV]  C=50, degree=3, gamma=0.005, kernel=linear, random_state=0, tol=1000, score=0.388, total=   0.0s
[CV] C=50, degre

[CV]  C=50, degree=3, gamma=0.005, kernel=poly, random_state=0, tol=1, score=0.619, total=   0.0s
[CV] C=50, degree=3, gamma=0.005, kernel=poly, random_state=0, tol=1 .
[CV]  C=50, degree=3, gamma=0.005, kernel=poly, random_state=0, tol=1, score=0.647, total=   0.0s
[CV] C=50, degree=3, gamma=0.005, kernel=poly, random_state=0, tol=1 .
[CV]  C=50, degree=3, gamma=0.005, kernel=poly, random_state=0, tol=1, score=0.624, total=   0.0s
[CV] C=50, degree=3, gamma=0.005, kernel=poly, random_state=0, tol=1 .
[CV]  C=50, degree=3, gamma=0.005, kernel=poly, random_state=0, tol=1, score=0.654, total=   0.0s
[CV] C=50, degree=3, gamma=0.005, kernel=poly, random_state=0, tol=1 .
[CV]  C=50, degree=3, gamma=0.005, kernel=poly, random_state=0, tol=1, score=0.647, total=   0.0s
[CV] C=50, degree=3, gamma=0.005, kernel=poly, random_state=0, tol=100 
[CV]  C=50, degree=3, gamma=0.005, kernel=poly, random_state=0, tol=100, score=0.388, total=   0.0s
[CV] C=50, degree=3, gamma=0.005, kernel=poly, random_

[CV]  C=50, degree=3, gamma=0.005, kernel=rbf, random_state=0, tol=10000, score=0.383, total=   0.0s
[CV] C=50, degree=3, gamma=0.005, kernel=rbf, random_state=0, tol=10000 
[CV]  C=50, degree=3, gamma=0.005, kernel=rbf, random_state=0, tol=10000, score=0.383, total=   0.0s
[CV] C=50, degree=3, gamma=0.005, kernel=rbf, random_state=0, tol=10000 
[CV]  C=50, degree=3, gamma=0.005, kernel=rbf, random_state=0, tol=10000, score=0.388, total=   0.0s
[CV] C=50, degree=3, gamma=0.005, kernel=rbf, random_state=0, tol=10000 
[CV]  C=50, degree=3, gamma=0.005, kernel=rbf, random_state=0, tol=10000, score=0.391, total=   0.0s
[CV] C=50, degree=3, gamma=0.005, kernel=rbf, random_state=0, tol=10000 
[CV]  C=50, degree=3, gamma=0.005, kernel=rbf, random_state=0, tol=10000, score=0.391, total=   0.0s
[CV] C=50, degree=3, gamma=0.005, kernel=rbf, random_state=0, tol=10000 
[CV]  C=50, degree=3, gamma=0.005, kernel=rbf, random_state=0, tol=10000, score=0.383, total=   0.0s
[CV] C=50, degree=3, gamma=0.

[Parallel(n_jobs=1)]: Done 10080 out of 10080 | elapsed:   22.9s finished


GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10, 50], 'degree': [1, 2, 3],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005],
                         'kernel': ['linear', 'poly', 'rbf'],
                         'random_state': [0, 42],
                         'tol': [1, 100, 1000, 10000, 10000, 1000000,
                                 10000000]},
             verbose=3)

In [14]:
# list the best parameters for this dataset, then list the best score
print(grid.best_params_)
print(grid.best_score_)

{'C': 5, 'degree': 1, 'gamma': 0.0001, 'kernel': 'linear', 'random_state': 0, 'tol': 1}
0.8393446302322971


In [15]:
# make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)

In [16]:
grid_df = pd.DataFrame({"Predictions": predictions, "Actual": y_test})
grid_df

,Predictions,Actual
280,0,0
434,1,1
39,1,1
417,0,0
584,1,1
...,...,...
424,0,0
101,0,0
199,0,0
398,1,1


In [17]:
# calculate classification report
# precision = true positive / (true positive + false positive)
# recall = true positive / (true positive + false negative)
# f1 = 2*((precision * recall) / (precision + recall)): best at 1, worst at 0
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.84      0.84      0.84       141
           1       0.73      0.73      0.73        82

    accuracy                           0.80       223
   macro avg       0.79      0.79      0.79       223
weighted avg       0.80      0.80      0.80       223



## Test the model

In [18]:
# loading the dataset
test_data = pd.read_csv("resources/test_transformed_2.csv")
test_data

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Family_Size,Age*Class,Fare_Per_Person,Sex_female,...,Title_Mrs,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_U,Deck_T
0,892,3,34,0,0,7.8292,0,102,7.829200,0,...,0,0,0,0,0,0,0,0,1,0
1,893,3,47,1,0,7.0000,1,141,3.500000,1,...,1,0,0,0,0,0,0,0,1,0
2,894,2,62,0,0,9.6875,0,124,9.687500,0,...,0,0,0,0,0,0,0,0,1,0
3,895,3,27,0,0,8.6625,0,81,8.662500,0,...,0,0,0,0,0,0,0,0,1,0
4,896,3,22,1,1,12.2875,2,66,4.095833,1,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,42,0,0,8.0500,0,126,8.050000,0,...,0,0,0,0,0,0,0,0,1,0
414,1306,1,39,0,0,108.9000,0,39,108.900000,1,...,0,0,0,1,0,0,0,0,0,0
415,1307,3,38,0,0,7.2500,0,114,7.250000,0,...,0,0,0,0,0,0,0,0,1,0
416,1308,3,21,0,0,8.0500,0,63,8.050000,0,...,0,0,0,0,0,0,0,0,1,0


In [19]:
# save the model if evaluation is promising
import joblib
filename = 'models/SVC.sav'
joblib.dump(grid, filename)

['models/SVC.sav']

In [20]:
# create variables for training and testing data
X_train = df[["Pclass", "Age", "Fare", "Family_Size", "Age*Class", "Fare_Per_Person", "Sex_female", "Title_Mr"]]
y_train = df["Survived"]

X_test = test_data[["Pclass", "Age", "Fare", "Family_Size", "Age*Class", "Fare_Per_Person", "Sex_female", "Title_Mr"]]

In [21]:
# Scale your data
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
from sklearn.svm import SVC
model = SVC(C=5, degree=1, gamma=0.0001, kernel='linear', random_state=0, tol=1)
model

SVC(C=5, degree=1, gamma=0.0001, kernel='linear', random_state=0, tol=1)

In [23]:
# fit(train) our model using the training data
model.fit(X_train_scaled, y_train)

SVC(C=5, degree=1, gamma=0.0001, kernel='linear', random_state=0, tol=1)

In [24]:
# make predictions with the hypertuned model
predictions = model.predict(X_test_scaled)

In [25]:
output = pd.DataFrame({'PassengerID': test_data.PassengerId, 'Survived': predictions})

In [26]:
# submission using grid hyperparameter tuned model
output.to_csv('submissions/Hunter_submission18.csv', index=False)